In [ ]:
import numpy as np
import os
import plotly.graph_objects as go

from chimera_fgo.util.kitti import process_kitti_gt, load_icp_results
from chimera_fgo.util.plot import plot_trajectories

%load_ext autoreload
%autoreload 2

In [ ]:
kitti_seq = '0034'
MAX_BIAS = 1
start_idx = 1550 if kitti_seq == '0028' else 0

In [ ]:
# Load results
run_name = 'fgo_-200m_1runs_blind_2023-01-18-2322'
results_path = os.path.join(os.getcwd(), '..', 'results', kitti_seq, run_name)
results_files = os.listdir(results_path)

In [ ]:
# Load ground truth
gtpath = os.path.join(os.getcwd(), '..', 'data', 'kitti', kitti_seq, 'oxts', 'data')
gt_enu, gt_Rs, gt_attitudes = process_kitti_gt(gtpath, start_idx=start_idx)

# Lidar data
data_path = os.path.join(os.getcwd(), '..', 'data', 'kitti', kitti_seq, 'icp')
lidar_Rs, lidar_ts, lidar_positions, lidar_covariances = load_icp_results(data_path, start_idx=start_idx)

In [ ]:
# Single run
results = np.load(os.path.join(results_path, results_files[0]))
graph_positions = results['positions']
gt_enu = gt_enu[:graph_positions.shape[0]]

# RMSE
rmse = np.sqrt(np.mean((graph_positions - gt_enu)**2))
print("RMSE:", rmse)

Error statistics

In [ ]:
np.set_printoptions(suppress=True, precision=3)

In [ ]:
# Nominal

false_alarms = 0
total_trials = 0
trajectory_false_alarms = 0

ss_start = 0  # Start of steady state
xyz_errors = []

for fname in results_files:
    results = np.load(os.path.join(results_path, fname))

    graph_positions = results['positions']
    gt_enu = gt_enu[:graph_positions.shape[0]]

    xyz_errors.append(gt_enu[ss_start:] - graph_positions[ss_start:])

    # False alarm
    qs = results['qs']
    threshold = results['threshold']
    false_alarms += np.sum(qs > threshold)
    total_trials += len(qs)
    if any(qs > threshold):
        trajectory_false_alarms += 1


print(f'Per trial false alarm rate: {false_alarms} of {total_trials}')
print(f'Per trajectory false alarm rate: {trajectory_false_alarms} of {len(results_files)}')

In [ ]:
xyz_errors = np.vstack(xyz_errors)

In [ ]:
print("mean: ", np.mean(xyz_errors, axis=0))
print("std: ", np.std(xyz_errors, axis=0))

Per trace error statistics

In [ ]:
kitti_seq = '0034'
run_name = 'fgo_100m_5runs_2023-01-19-0707'

results_path = os.path.join(os.getcwd(), '..', 'results', kitti_seq, run_name)
results_files = os.listdir(results_path)

start_idx = 1550 if kitti_seq == '0028' else 0
gtpath = os.path.join(os.getcwd(), '..', 'data', 'kitti', kitti_seq, 'oxts', 'data')
gt_enu, gt_Rs, gt_attitudes = process_kitti_gt(gtpath, start_idx=start_idx)

xyz_errors = []

for fname in results_files:
    results = np.load(os.path.join(results_path, fname))

    graph_positions = results['positions']
    gt_enu = gt_enu[:graph_positions.shape[0]]

    xyz_errors.append(gt_enu - graph_positions)

xyz_errors = np.vstack(xyz_errors)

# print("rmse: ", np.sqrt(np.mean(xyz_errors**2)))
# print("std: ", np.std(xyz_errors))
# print("max: ", np.max(xyz_errors))

norm_errors = np.linalg.norm(xyz_errors, axis=1)
print("mean: ", np.mean(norm_errors))
print("std: ", np.std(norm_errors))
print("max: ", np.max(norm_errors))